In [1]:
import healpy as hp
import numpy as np

In [3]:
randMap = hp.sphtfunc.synfast(cls=np.ones(6144),nside=1024);

In [9]:
!python3 kSZ_reconstruct.py --ksz_reconstruction

  Creating pywig tables...
/home/wurdahl/anaconda3/lib/python3.9/site-packages/healpy/fitsfunc.py:368: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  warnings.warn(
Traceback (most recent call last):
  File "/mnt/c/Users/wurda/Documents/GitHub/kSZfromPC/kSZ_reconstruct.py", line 179, in <module>
    tau_map = -1.0*getMap("N_uncertain", NSIDE=NSIDE_WORKING) #
  File "/mnt/c/Users/wurda/Documents/GitHub/kSZfromPC/common.py", line 97, in getMap
    _map = getFits(MAPS_OUTPUT_DIR+name+".fits")
  File "/mnt/c/Users/wurda/Documents/GitHub/kSZfromPC/common.py", line 89, in getFits
    data = hp.fitsfunc.read_map(filepath)
  File "/home/wurdahl/anaconda3/lib/python3.9/site-packages/healpy/fitsfunc.py", line 378, in read_map
    filename = pf.open(filename, memmap=me